# Table of contents
*  [Introduction](#section1) 
*  [Libraries](#section2) 
*  [Read in the data](#section3)
*  [Feature engineering](#section4)
    - [Missing values](#section5)
        - [Numerical columns](#section6)
        - [Text columns](#section7)
    - [Adding new features](#section8)
    - [Removing columns](#section9)
        - [Irrelevant columns](#section10)
        - [Columns that leak sale information](#section11)
    - [Categorical features](#section12)
*  [Feature selection](#section13)
    - [Data preparation](#section14)
    - [1. Correlations](#section15)
    - [2. Univariate Selection](#section16)
    - [3. Recursive Feature Elimination](#section17)
    - [4. Extra Trees classifier - Feature Importance](#section18)
*  [Modeling and testing](#section19)

by @antosnj

-----
<a id='section1'></a>
# Introduction
This kernel aims to predict house sale prices using different linear regression models based on four feature selection techniques: Simple correlations, Univariate Selection, Recursive Feature Elimination and Extra Trees Classifier. The dataset consists of housing data for the city of Ames, Iowa, United States from 2006 to 2010. 

- Info about why the data was collected can be found at http://jse.amstat.org/v19n3/decock.pdf.
- Info about the dataset columns at http://jse.amstat.org/v19n3/decock/DataDocumentation.txt

----
<a id='section2'></a>
# Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_val_predict

import warnings
warnings.filterwarnings('ignore')

---
<a id='section3'></a>
# Read in the data

In [ ]:
housing_data = pd.read_csv('../input/ameshousing/AmesHousing/AmesHousing.tsv', delimiter="\t")
print(housing_data.shape)
housing_data.head()

---
<a id='section4'></a>
# Feature engineering

<a id='section5'></a>
## Missing values
---
<a id='section6'></a>
### Numerical columns
The data contains 2930 observations. Let's start out by analyzing which numerical columns have less than 5% of missing values.

In [ ]:
numerical_cols = housing_data.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64']).columns
null_values = housing_data[numerical_cols].isnull().sum()
less_than_5pct = null_values[(null_values != 0) & (null_values < 0.05*2930)]
most_common = housing_data[less_than_5pct.index].mode()

print(less_than_5pct)
print("\nMOST COMMON VALUES:\n", most_common)

 and fill those out with the most popular value of each column.

In [ ]:
housing_data.loc[:, less_than_5pct.index] = housing_data.loc[:, less_than_5pct.index].fillna(most_common.loc[0])
housing_data[less_than_5pct.index].isnull().sum()

---
It turns out that there are only two columns that contain more than 5% of missing values in the entire dataset. In particular, they're missing a 16.7% and a 5.4% of the total number of rows respectively, as seen below.

In [ ]:
more_than_5pct = null_values[null_values >= 0.05*2930]
cols = more_than_5pct.index.tolist() + ['SalePrice']

print("\nPERCENTAGE OF MISSING VALUES:")
print(more_than_5pct/2930)
print("\nCORRELATION:")
print(housing_data[cols].corr())

'Lot Frontage' has more than 15% of missing values, shows a low correlation with 'SalePrice' (the target column we want to predict) and won't help us create new features a priori, so I'll drop the column. However, 'Garage Yr Blt' shows more correlation and might be useful later on, so let's fill out the 5.4% missing values with the most common value in the column as well.

In [ ]:
housing_data.drop(columns='Lot Frontage', inplace=True)
most_common_value = housing_data['Garage Yr Blt'].mode()
housing_data.loc[:, 'Garage Yr Blt'] = housing_data.loc[:, 'Garage Yr Blt'].fillna(most_common_value)

<a id='section7'></a>
### Text columns
Similarly, let's see which text columns have more than 5% of missing values.

In [ ]:
text_cols = housing_data.select_dtypes(include=['object']).columns
null_values = housing_data[text_cols].isnull().sum()
more_than_5pct = null_values[null_values > 0.05*2930]
more_than_5pct

Given the number of missing values in 'Alley', 'Fireplace Qu', 'Pool QC', 'Fence' and 'Misc Feature', I will drop them. For 'Garage Type', 'Garage Finish', 'Garage Qual' and 'Garage Cond', it may work best if we just drop the rows where the missing values happen, since there're not as many.

In [ ]:
housing_data.drop(['Alley', 'Fireplace Qu', 'Pool QC', 'Fence', 'Misc Feature'], axis=1, inplace=True)
housing_data.dropna(subset=['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond'], inplace=True)

<a id='section8'></a>
## Adding new features
The following columns could be combined to get new useful features:
- 'Year Built' - 'Yr Sold': Compute the age (years) of the house when at the time it was sold.
- 'Year Remod/Add' - 'Yr Sold': Compute the years passed between the remodel date and the year it was sold.

After creating the new features, we can drop them.

In [ ]:
housing_data['age_when_sold'] = housing_data['Yr Sold'] - housing_data['Year Built']
housing_data['years_remodeled'] = housing_data['Yr Sold'] - housing_data['Year Remod/Add']

for col in ['Year Built', 'Year Remod/Add']:
    housing_data.drop(col, axis=1, inplace=True)
    
print(housing_data['age_when_sold'].value_counts())
print(housing_data['years_remodeled'].value_counts())

Looks like there're some negative values. Since we're representing age, we need to get rid of the few rows where those happen. 

In [ ]:
negative_age_index = housing_data.loc[housing_data['age_when_sold'] < 0].index.tolist()
negative_remodeled_index = housing_data.loc[housing_data['years_remodeled'] < 0].index.tolist()

negative_rows = negative_age_index + negative_remodeled_index
housing_data.drop(negative_rows, axis=0, inplace=True)

---
<a id='section9'></a>
## Removing columns

<a id='section10'></a>
### Irrelevant columns
Some of the columns are not going to be useful for the model. In particular, looking at the documentation we could exclude the following:
- 'Order': Observation number
- 'PID': Parcel identification number.

In [ ]:
housing_data.drop(['Order', 'PID'], axis=1, inplace=True)

<a id='section11'></a>
### Columns that leak sale information
These columns contain information about the sale itself. The new houses our model will be used for to predict prices in the future won't have that information, since they haven't been sold yet, so we can't use it as part of our feature selection. In the dataset, these include:

- 'Sale Type': Type of sale
- 'Yr Sold': Year Sold (YYYY)
- 'Mo Sold': Month Sold (MM)
- 'Sale Condition': Condition of sale

In [ ]:
housing_data.drop(['Sale Type', 'Yr Sold', 'Mo Sold', 'Sale Condition'], axis=1, inplace=True)

# See how the data looks like after cleaning and transforming features
housing_data.head()

<a id='section12'></a>
## Categorical features
Candidates to be converted to categorical columns are the nominal ones. Looking at the documentation, they are the following:

- PID: Parcel identification number
- MS Zoning: Identifies the general zoning classification of the sale
- Street: Type of road access to property
- Alley: Type of alley access to property
- Lot Shape: General shape of property
- Land Contour: Flatness of the property
- Utilities: Type of utilities available
- Lot Config: Lot configuration
- Land Slope: Slope of property
- Neighborhood: Physical locations within Ames city limits (map available)
- Condition 1: Proximity to various conditions
- Condition 2: Proximity to various conditions (if more than one is present)
- Bldg Type: Type of dwelling
- House Style: Style of dwelling
- Roof Style: Type of roof
- Roof Matl: Roof material
- Exterior 1st: Exterior covering on house
- Exterior 2nd: Exterior covering on house (if more than one material)
- Mas Vnr Type: Masonry veneer type
- Foundation: Type of foundation
- Heating: Type of heating
- Central Air: Central air conditioning
- Garage Type: Garage location
- Misc Feature: Miscellaneous feature not covered in other categories
- Sale Type: Type of sale
- Sale Condition: Condition of sale

Some of them have been dropped earlier, so we don't have to consider them anymore. Let's also see how many unique values these columns have, since if they're too many, then too many columns will need to be added back to the data frame if we dummy the code.

In [ ]:
categorical_cols = ['PID', 'MS Zoning', 'Street', 'Alley', 'Lot Shape', 'Land Contour', 
        'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1', 
        'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 
        'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Foundation', 'Heating', 
        'Central Air', 'Garage Type', 'Misc Feature', 'Sale Type', 'Sale Condition']

# Check dropped columns
print("DROPPED COLS:\n")
dropped_cols = []
for col in categorical_cols:
    if col not in housing_data.columns:
        print(col)
        dropped_cols.append(col)

# Remove them from 'categorical_cols'
for col in dropped_cols:
    categorical_cols.remove(col)

# Check number of unique values
housing_data[categorical_cols].describe().loc['unique'].sort_values(ascending=False)

Based on the results and to set a cutoff, let's exclude 'Neighborhood', 'Exterior 1st' and 'Exterior 2nd', which have more than 10 unique values.

In [ ]:
for col in ['Neighborhood', 'Exterior 1st', 'Exterior 2nd']:
    categorical_cols.remove(col)
    
for col in categorical_cols:
    housing_data[col] = housing_data[col].astype('category')

---
We don't want columns that have a few unique values but too many values in the column belong to a specific category either (say more than 90% of the total number of values in the column), since columns with low variance mean no variability in the data for the model to capture.

In [ ]:
low_variance_cols = []

for col in categorical_cols:
    col_value_count = housing_data[col].value_counts()
    pct = col_value_count/sum(col_value_count)
    
    # Find any column that has one value representing 90% of the total number of values
    if max(pct) > 0.9:
        print(pct.name)
        low_variance_cols.append(pct.name)

Let's drop those from 'categorical_cols'.

In [ ]:
for col in low_variance_cols:
    categorical_cols.remove(col)

# See how 'housing_data' final categorical columns look like 
housing_data[categorical_cols].head()

Finally, let's create the new dummy variables using the categorical columns.

In [ ]:
print("Before:", housing_data.shape)

for col in categorical_cols:
    dummy_col = pd.get_dummies(housing_data[col])
    housing_data = pd.concat([housing_data, dummy_col], axis=1)
    housing_data.drop(col, axis=1, inplace=True)

print("After:", housing_data.shape)
housing_data.head()

51 new columns were successfully added to the data frame. Before proceeding to feature selection, let's take another look at any correlations with 'SalePrice' higher than 0.5

In [ ]:
corr = housing_data.corr()['SalePrice'].sort_values()
corr[corr > 0.5]

---
<a id='section13'></a>
# Feature Selection
For feature selection, I am going to take 4 different approaches and contrast their results, testing a linear regression model. To start out with, I will select 10 features, but I will experiment with different number of features later on when testing different models:
- Correlations
- Univariate selection
- Recursive Feature Elimination
- Principal Component Analysis
- Feature Importance

For reference about some of the techniques I've used see https://machinelearningmastery.com/feature-selection-machine-learning-python/.

<a id='section14'></a>
## Data preparation

In [ ]:
# Select numerical features
numerical_data = housing_data.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
X = numerical_data.loc[:, numerical_data.columns != 'SalePrice']

# Target column
y = numerical_data['SalePrice']

col_names = housing_data.columns.values

<a id='section15'></a>
## 1. Correlations

In [ ]:
# Sorted correlations with target column 'SalePrice'
numerical_data = housing_data.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
sorted_corrs = numerical_data.corr()['SalePrice'].sort_values()
strong_corrs_features = sorted_corrs[sorted_corrs > 0.5]

fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(housing_data[strong_corrs_features.index].corr())

In [ ]:
print("\nSTRONG CORRELATIONS WITH 'SalePrice':\n\n", strong_corrs_features)

<a id='section16'></a>
## 2. Univariate Selection
For this technique, let's consider the chi squared (chi^2) statistical test for non-negative features to select 10 of the best features.

In [ ]:
# Feature extraction
selector = SelectKBest(score_func=chi2, k=10)
fit = selector.fit(X, y)

# Summarize scores
transformed_features = fit.transform(X)

mask = selector.get_support() 
new_features = [] # The list of K best features

for bool, feature in zip(mask, col_names):
    if bool:
        new_features.append(feature)

# df with 10 best features using univariate selection
data_univ_selection = pd.DataFrame(transformed_features, columns=new_features)

print("\n10 BEST FEATURES:\n\n", new_features)
np.set_printoptions(precision=3)
print("\nSCORES:\n\n", fit.scores_)

<a id='section17'></a>
## 3. Recursive Feature Elimination

In [ ]:
# Feature extraction
model = LinearRegression()
rfe = RFE(model, 10)
fit = rfe.fit(X, y)

indexes = [i for i, x in enumerate(fit.support_) if x]
feature_names = X.columns[indexes].tolist()

# df with 10 best features using RFE
data_RFE = X[feature_names]

# Summarize components
print("Num Features:", fit.n_features_)
print("\nSelected Features:\n", fit.support_) 
print("\nFeature Ranking:\n", fit.ranking_)
print("\nFeature names:\n", feature_names)

<a id='section18'></a>
## 4. Extra Trees classifier - Feature Importance

In [ ]:
# Feature extraction
model = ExtraTreesClassifier()
model.fit(X, y)

# Select 10 best features
importances = model.feature_importances_.tolist()
max_importances = np.sort(importances)[::-1][0:10]

best_indexes = []
for x in max_importances:
    best_indexes.append(importances.index(x))
    
best_features = X.columns[best_indexes].tolist()

# df with 10 best features using univariate selection
data_ET = X[best_features]

print("\n10 BEST FEATURES:\n\n", best_features)
print("\nIMPORTANCE VALUES:\n\n", max_importances)

---
<a id='section19'></a>
# Modeling and Testing
Let's put it all together in a function in such a way we can test different models easily, starting with a feature selection function that gathers all feature selection techniques discussed before.

In [ ]:
# ----------------------------------------------------------
#                   FEATURE SELECTION
# ----------------------------------------------------------
# Parameters:
#
# df: data frame containing data
# target_col: target column 
# num_features: number of features to be used
# technique: technique to be used for feature selection:
#            'CORR': Select features with correlation with target column higher than 0.5
#            'US': Univariate Selection
#            'RFE': Recursive Feature Elimination
#            'ET': Extra Trees classifier - Feature importance
# 
# Returns:
#
# df_technique (DataFrame): training data containing selected features using the input technique
# best_features (list): names of selected features


def feature_selection(df, target_col, num_features, technique):
    
    # Select numerical features
    numerical_data = df.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
    X = numerical_data.loc[:, numerical_data.columns != target_col]

    # Target column
    y = df[target_col]
    
    # Select features based on input technique:
    if technique == 'CORR':
        sorted_corrs = numerical_data.corr()[target_col].sort_values()
        best_features = sorted_corrs[(sorted_corrs > 0.5) & (sorted_corrs.index != target_col)].index.tolist()
        
        df_technique = X[best_features]
        
        
    elif technique == 'US':
        selector = SelectKBest(score_func=chi2, k=num_features)
        fit = selector.fit(X, y)
        transformed_features = fit.transform(X)

        mask = selector.get_support() 
        col_names = df.columns.values
        best_features = [] # The list of K best features
    
        for bool, feature in zip(mask, col_names):
            if bool:
                best_features.append(feature)

        df_technique = pd.DataFrame(transformed_features, columns=best_features)
        
        
    elif technique == 'RFE':
        model = LinearRegression()
        rfe = RFE(model, num_features)
        fit = rfe.fit(X, y)

        indexes = [i for i, x in enumerate(fit.support_) if x]
        best_features = X.columns[indexes].tolist()

        df_technique = X[best_features]

    
    elif technique == 'ET':
        model = ExtraTreesClassifier()
        model.fit(X, y)
        
        importances = model.feature_importances_.tolist()
        max_importances = np.sort(importances)[::-1][0:num_features]

        best_indexes = []
        for x in max_importances:
            best_indexes.append(importances.index(x))

        best_features = X.columns[best_indexes].tolist()
        df_technique = X[best_features]
        
        
    return(df_technique, best_features)

---
Next, let's define a train_and_test function based on a linear regression model and _k_-fold cross validation, that makes predictions and returns the corresponding RMSE value depending on the number of features, number of folds and the feature selection technique.

In [ ]:
def train_and_test(data, target_col, num_features, k, feature_sel_technique):
    
    X, features = feature_selection(data, target_col, num_features, feature_sel_technique)
    y = data[target_col]
    
    # If number of folds equals 1
    if k == 1:
        data = data.sample(frac=1, )
        
        X_train = X[:1460]
        y_train = y[:1460]
        X_test = X[1460:]
        y_test = y[1460:]
        
        lir = LinearRegression()
        
        lir.fit(X_train, y_train)
        new_prices_1 = lir.predict(X_test)        
        mse_one = mean_squared_error(y_test, new_prices_1)
        rmse_one = np.sqrt(mse_one)
        
        lir.fit(X_test, y_test)
        new_prices_2 = lir.predict(X_train)        
        mse_two = mean_squared_error(y_train, new_prices_2)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        
        return(avg_rmse, new_prices_1, features)
    


    # Model
    # Train
    lir = LinearRegression()
    lir.fit(X, y)
    kf = KFold(n_splits=k, shuffle=True, random_state=1)
    
    # Predict
    new_prices = cross_val_predict(lir, X, y, cv=kf)

    # Test - Evaluate RMSE
    mses = cross_val_score(lir, X, y, scoring='neg_mean_squared_error', cv=kf)
    avg_rmse = np.mean(np.sqrt(np.absolute(mses)))
    
    return(avg_rmse, new_prices, features)

---
Let's test different linear regression models by changing the number of features to be selected, the technique to be used to select such features, and the parameter _k_ to control the type of (_k_-fold) cross validation that occurs, and save the one that returns the lowest RMSE to 'best_rmse', and the one that returns the lowest RMSLE to 'best_rmsle'.

In [ ]:
num_features = range(1, 30)
n_folds = range(1, 11)
techniques = ['CORR', 'US', 'RFE', 'ET']
target_col = 'SalePrice'

# Define a RMSLE function
def rmsle(y_pred, y_test) : 
    assert len(y_test) == len(y_pred)
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))

current_rmse = math.inf
current_rmsle = math.inf
best_rmse = {}
best_rmsle = {}
y_test = housing_data[1460:]

for k in n_folds:
    for technique in techniques:
        for n_features in num_features:
            avg_rmse, new_prices, features = train_and_test(housing_data, target_col, n_features, k, technique)
            
            if k == 1:
                rmsle_v = rmsle(new_prices, y_test[target_col])
            else:
                rmsle_v = rmsle(new_prices[1460:], y_test[target_col])

            
            if avg_rmse < current_rmse:
                best_rmse['RMSE'] = avg_rmse
                best_rmse['new_prices'] = new_prices
                best_rmse['features'] = features  
                best_rmse['technique'] = technique  
                best_rmse['num_features'] = n_features  
                best_rmse['n_folds'] = k
                
                current_rmse = avg_rmse
                
            if rmsle_v < current_rmsle:
                best_rmsle['RMSLE'] = rmsle_v
                best_rmsle['new_prices'] = new_prices
                best_rmsle['features'] = features  
                best_rmsle['technique'] = technique  
                best_rmsle['num_features'] = n_features  
                best_rmsle['n_folds'] = k
                
                current_rmsle = rmsle_v
                
print(best_rmse)
print(best_rmsle)

### Best results:
---
- **RMSLE: 0.160765**
    - _Feature selection technique_: Extra Trees Classifier - Feature Importance
    - _Number of folds cross validation (k)_: 1
    - _Number of features used_: 21
    - _Features used/selected_: ['Bsmt Unf SF', 'Gr Liv Area', 'Lot Area', 'Garage Area', 'Total Bsmt SF', 'years_remodeled', '1st Flr SF', 'age_when_sold', 'Garage Yr Blt', 'BsmtFin SF 1', 'Wood Deck SF', 'Open Porch SF', 'TotRms AbvGrd', 'Mas Vnr Area', 'Overall Qual', 'Overall Cond', 'Bedroom AbvGr', '2nd Flr SF', 'Fireplaces', 'MS SubClass', 'Bsmt Full Bath']
    
    
- **RMSE: 31293.8494**
    - _Feature selection technique_: Extra Trees Classifier - Feature Importance
    - _Number of folds cross validation (k)_: 8
    - _Number of features used_: 27
    - _Features used/selected_: ['Lot Area', 'Garage Area', '1st Flr SF', 'Gr Liv Area', 'Bsmt Unf SF', 'years_remodeled', 'Total Bsmt SF', 'age_when_sold', 'Garage Yr Blt', 'BsmtFin SF 1', 'Open Porch SF', 'Wood Deck SF', 'TotRms AbvGrd', 'Mas Vnr Area', 'Overall Qual', '2nd Flr SF', 'Overall Cond', 'Bedroom AbvGr', 'MS SubClass', 'Fireplaces', 'Bsmt Full Bath', 'Enclosed Porch', 'Garage Cars', 'BsmtFin SF 2', 'Full Bath', 'Half Bath', 'Screen Porch']
---
**Note:** For iterations where less features were used, as an example, for the case of 14 features it returned a RMSLE of 0.171812 (k=1 folds) and a RMSE of 32888.7108 (k=8 folds) using Recursive Feature Elimination (RFE).